# Chương này bao gồm những kiến thức chính:

* Đếm từ và tần số xuất hiện để phân tích ngữ nghĩa
* Dự đoán xác suất xuất hiện từ với Zip'f Law
* Biểu diễn vecto từ và làm thế nào để bắt đầu sử dụng chúng
* Tìm document liên quan từ một corpus sử dụng tần số document nghịch đảo
* Ước tính sự giống nhau của document với cosine và Okapi BM25

Trong chương này, chúng ta sẽ xem xét 3 cách mạnh mẽ để biểu diễn từ và tầm quan trọng của chúng trong 1 document:
* <i>Bags of words</i>: Vector hóa số từ hoặc tần suất
* <i>Bags of n-grams</i>: Đếm số cặp từ (bigram), triplets (trigrams), ...
* <i>TF-IDF vectors</i>: Chấm điểm từ, đại diện cho mức độ quan trọng của chúng.

QUAN TRỌNG: TF-IDF là term frequenct times inverse document frequency.<br/>
Team frequencies là đếm mỗi từ trong document, cái mà bạn đã học ở chương trước. Inverse document frequency nghĩa là bạn sẽ chia mỗi số từ đó cho số từ trong tài liệu.
    

Mỗi một kỹ thuật có thể được áp dụng riêng rẽ hoặc như một phần của pipeline NLP. Đây là mô hình thống kê dựa trên frequency (tần số) xuất hiện. Phần sau của quyển sách này, bạn sẽ có cách khác để tìm hiểu sâu hơn về mối quan hệ giữa các từ và mô hình giữa chúng.

Nhưng máy học NLP "shallow" (nông cạn) là mạnh mẽ và hữu ích cho nhiều ứng dụng thực tiễn như lọc spam và sentimen analysis (phân tích cảm xúc).

## Bag of words

Ở chương trước, bạn đã tạo một mô hình không gian vecto đầu tiên của một văn bản. Bạn sử dụng one-hot encoding cho mỗi từ và rồi tổ hợp tất cả các vector với một phép toán OR (hoặc sum) để tạo một vector biểu diễn cho 1 text. Và vecto bag-of-words nhị phân này cần một lượng index lớn để biểu diễn document khi load vào trong 1 cấu trúc như là Pandas DataFrame.

Ví dụ về đếm số lần xuất hiện của từ:

In [1]:
from nltk.tokenize import TreebankWordTokenizer
sentence = """The faster Harry got to the store, the faster Harry,
... the faster, would get home."""
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(sentence.lower())

In [2]:
tokens

['the',
 'faster',
 'harry',
 'got',
 'to',
 'the',
 'store',
 ',',
 'the',
 'faster',
 'harry',
 ',',
 '...',
 'the',
 'faster',
 ',',
 'would',
 'get',
 'home',
 '.']

Với list đơn của bạn, bạn muốn lấy unique words từ document và đếm chúng. Dictionary trong python phục vụ tốt cho mục đích này, và bởi vì bạn muốn đếm số từ, bạn có thể sử dụng <b>Counter</b>, như bạn đã làm ở chương trước:

In [3]:
from collections import Counter
bag_of_words = Counter(tokens)
bag_of_words

Counter({'the': 4,
         'faster': 3,
         'harry': 2,
         'got': 1,
         'to': 1,
         'store': 1,
         ',': 3,
         '...': 1,
         'would': 1,
         'get': 1,
         'home': 1,
         '.': 1})

Như dictionary Python, thứ tự các key bị xáo trộn. Sự sắp xếp mới này tối ưu cho lưu trữ, cập nhật và phục hồi, không thích hợp hiển thị. Thông tin về thứ tự ban đầu của các từ bị loại bỏ.

NOTE: Một <b>collections.Counter</b> là một đối tượng collection không thứ tự. Hơn nữa, gọi là một bag hoặc multiset. Phụ thuộc vào nền tảng của bạn và phiên bản Python, bạn có thể thấy rằng một <b>Counter</b> được hiển thị trong một thứ tự hợp lý, giống như thứ tự từ vựng hoặc thứ tự xuất hiện trong tokens của bạn. Nhưng cũng giống như dict chuẩn của Python, bạn không thể tin vào sắp xếp của tokens (keys) trong một <b>Counter</b>

Đối với document ngắn như thế này, bag of words không thứ tự cũng bao gồm rất nhiều thông tin quan trọng về mục đích và ngữ nghĩa. Và thông tin trong bag of word là đủ để thực hiện 1 số thứ hữu ích như detect sapm, phân tích cảm xúc. Nó có thể như 1 cái túi, nhưng nó chứa đầy ý nghĩa và thông tin. Vì vậy, sắp xếp những từ này theo một trật tự dễ dàng hơn để hiểu. Đối tượng <b>Counter</b> có một phương thức thực hiện, <b>most_common</b> cho mục đích này:

In [4]:
bag_of_words.most_common(4)

[('the', 4), ('faster', 3), (',', 3), ('harry', 2)]

Mặc định, most_common() liệt kê tất cả các tokens phổ biến nhất, nhưng bạn có thể giới hạn số lượng list ra bằng cách thêm số vào như số 4 bên trên.

Đặc biệt, số lần một từ xuất hiện trong 1 document được gọi là <b>tearm frequency</b> (tần số), viết tắt là TF. <br/>
Vì vậy 4 term hoặc token đầu của bạn là  “the,” “,”, “harry,” and “faster.”. Tuy nhiên từ "the" và ký hiệu "," là không quan trọng trong ý nghĩa của document. Và những token uninfomative (không thông tin) xuất hiện nhiều. Vì vậy, trong ví dụ này, chúng ta sẽ loại bỏ chúng, cùng với một danh sách stop word và dấu chấm câu. Điều này không luôn luôn loại ở được mọi trường hợp, nhưng cho đến thời điểm này thì nó là ví dụ đơn giản. Bạn thấy rằng các từ "harry" và "faster" là các token hàng đầu ở trong vector TF của bạn (bag of word.<br/>
Nào, hãy tính toán term frequency của "harry" từ đối tượng <b>Counter</b> bạn định nghĩa ở trên:

In [5]:
times_harry_appears=bag_of_words["harry"] # số lần xuất hiện của từ harry
print(times_harry_appears)
num_unique_words = len(bag_of_words) # số unique token nguồn
print(num_unique_words)
tf = times_harry_appears/num_unique_words
print(round(tf,4))

2
12
0.1667


Hãy dừng lại vài giây và tìm hiểu sâu hơn một chút về team frequency, một từ mà chúng ta sử dụng thường xuyên ở cuốn sách này.<br/>
Khi nói bạn đã tìm được từ "dog" 3 lần ở document A và 100 lần ở document B. Rõ ràng "dog" là quan trọng hơn trong document B. Nhưng khoan. Hãy nói bạn tìm ở document A là một email 30 từ và document B là tiểu thuyết Chiến tranh và hòa bình (War and Peace) (xấp xỉ 580,000 từ). Phân tích như sau:<br/>

$$TF("dog",document_A)=3/30=.1$$
$$TF("dog",document_B)=100/580000=0.0017$$

Bây giờ bạn có thể thấy rằng các phân tích mô tả một cái gì đó về 2 văn bản và mối quan hệ giữa chúng với từ "dog". Vì vậy, thay vì đếm số từ thô để mô tả document của bạn trong một corpus, bạn có thể chuẩn hóa với term frequencties. Cũng như thế, bạn có thể tính toán mỗi từ và lấy được độ quan trọng tương đối của từ đó với document.Bạn đã thực hiện một số tiến bộ lớn trong việc số hóa văn bản, vượt ra ngoài chỉ sự hiện diện hay vắng mặt của một từ.

Loại bỏ stop word:

In [6]:
import nltk
nltk.download('stopwords', quiet=True)

True

In [7]:
tokens

['the',
 'faster',
 'harry',
 'got',
 'to',
 'the',
 'store',
 ',',
 'the',
 'faster',
 'harry',
 ',',
 '...',
 'the',
 'faster',
 ',',
 'would',
 'get',
 'home',
 '.']

In [8]:
stopwords = nltk.corpus.stopwords.words('english')
tokens = [x for x in tokens if x not in stopwords]
kite_counts = Counter(tokens)
kite_counts

Counter({'faster': 3,
         'harry': 2,
         'got': 1,
         'store': 1,
         ',': 3,
         '...': 1,
         'would': 1,
         'get': 1,
         'home': 1,
         '.': 1})

In [9]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## Vectorizing

Bạn đã có thể chuyển đổi văn bản của bạn thành số ở mức cơ bản. Nhưng bạn vẫn chỉ lưu trữ chúng ở dictionary, tiếp theo chúng ta sẽ tạo một vector của số từ.

In [10]:
tokens

['faster',
 'harry',
 'got',
 'store',
 ',',
 'faster',
 'harry',
 ',',
 '...',
 'faster',
 ',',
 'would',
 'get',
 'home',
 '.']

In [11]:
document_vector = []
doc_length = len(tokens)
for key, value in kite_counts.most_common():
    document_vector.append(value/doc_length)

In [12]:
document_vector

[0.2,
 0.2,
 0.13333333333333333,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667,
 0.06666666666666667]

Có thể tăng tốc xử lý bằng numpy

colections của các từ (bộ sưu tập các từ) trong vocabulary được gọi là <i>lexicon</i>

In [13]:
docs = ["The faster Harry got to the store, the faster and faster Harry would get home."]
docs.append("Harry is hairy and faster than Jill.")
docs.append("Jill is not as hairy as Harry.")

In [14]:
docs

['The faster Harry got to the store, the faster and faster Harry would get home.',
 'Harry is hairy and faster than Jill.',
 'Jill is not as hairy as Harry.']

In [15]:
doc_tokens = []
for doc in docs:
    doc_tokens += [sorted(tokenizer.tokenize(doc.lower()))]

In [16]:
len(doc_tokens[0])

17

In [17]:
doc_tokens

[[',',
  '.',
  'and',
  'faster',
  'faster',
  'faster',
  'get',
  'got',
  'harry',
  'harry',
  'home',
  'store',
  'the',
  'the',
  'the',
  'to',
  'would'],
 ['.', 'and', 'faster', 'hairy', 'harry', 'is', 'jill', 'than'],
 ['.', 'as', 'as', 'hairy', 'harry', 'is', 'jill', 'not']]

In [18]:
all_doc_tokens = sum(doc_tokens, [])

In [24]:
all_doc_tokens

[',',
 '.',
 'and',
 'faster',
 'faster',
 'faster',
 'get',
 'got',
 'harry',
 'harry',
 'home',
 'store',
 'the',
 'the',
 'the',
 'to',
 'would',
 '.',
 'and',
 'faster',
 'hairy',
 'harry',
 'is',
 'jill',
 'than',
 '.',
 'as',
 'as',
 'hairy',
 'harry',
 'is',
 'jill',
 'not']

In [19]:
len(all_doc_tokens)

33

In [20]:
all_doc_tokens

[',',
 '.',
 'and',
 'faster',
 'faster',
 'faster',
 'get',
 'got',
 'harry',
 'harry',
 'home',
 'store',
 'the',
 'the',
 'the',
 'to',
 'would',
 '.',
 'and',
 'faster',
 'hairy',
 'harry',
 'is',
 'jill',
 'than',
 '.',
 'as',
 'as',
 'hairy',
 'harry',
 'is',
 'jill',
 'not']

In [21]:
lexicon = sorted(set(all_doc_tokens))

In [22]:
lexicon

[',',
 '.',
 'and',
 'as',
 'faster',
 'get',
 'got',
 'hairy',
 'harry',
 'home',
 'is',
 'jill',
 'not',
 'store',
 'than',
 'the',
 'to',
 'would']

In [23]:
len(lexicon)

18

Mỗi vector trong 3 document của bạn sẽ cần có 18 giá trị.

In [25]:
from collections import OrderedDict
zero_vector = OrderedDict((token, 0) for token in lexicon)
zero_vector

OrderedDict([(',', 0),
             ('.', 0),
             ('and', 0),
             ('as', 0),
             ('faster', 0),
             ('get', 0),
             ('got', 0),
             ('hairy', 0),
             ('harry', 0),
             ('home', 0),
             ('is', 0),
             ('jill', 0),
             ('not', 0),
             ('store', 0),
             ('than', 0),
             ('the', 0),
             ('to', 0),
             ('would', 0)])

Bây giờ chúng ta sẽ copy giá trị của các vecto cơ sở, cập nhật giá trị của mỗi vecto của document, và lưu chúng trong 1 mảng.

In [31]:
import copy
doc_vectors = []
for doc in docs:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_counts = Counter(tokens)
    for key, value in token_counts.items():
        vec[key] = value/len(lexicon)
    doc_vectors.append(vec)
doc_vectors

[OrderedDict([(',', 0.05555555555555555),
              ('.', 0.05555555555555555),
              ('and', 0.05555555555555555),
              ('as', 0),
              ('faster', 0.16666666666666666),
              ('get', 0.05555555555555555),
              ('got', 0.05555555555555555),
              ('hairy', 0),
              ('harry', 0.1111111111111111),
              ('home', 0.05555555555555555),
              ('is', 0),
              ('jill', 0),
              ('not', 0),
              ('store', 0.05555555555555555),
              ('than', 0),
              ('the', 0.16666666666666666),
              ('to', 0.05555555555555555),
              ('would', 0.05555555555555555)]),
 OrderedDict([(',', 0),
              ('.', 0.05555555555555555),
              ('and', 0.05555555555555555),
              ('as', 0),
              ('faster', 0.05555555555555555),
              ('get', 0),
              ('got', 0),
              ('hairy', 0.05555555555555555),
              ('harry', 0.05

### Không gian vector

In [32]:
import math
def consine_sim(vec1, vec2):
    """ Let's convert our dictionaries to lists for easier matching """
    vec1 = [val for val in vec1.values()]
    vec2 = [val for val in vec2.values()]
    
    dot_prod = 0
    for i,v in enumerate(vec1):
        dot_prod += v*vec2[i]
    
    mag_1 = math.sqrt(sum([x**2 for x in vec1]))
    mag_2 = math.sqrt(sum([x**2 for x in vec2]))
    
    return dot_prod/(mag_1*mag_2)

Nói về consine semilarity giữa 2 vector

# Zipf's Law

Zipf’s law states that given some corpus of natural language utterances, the
frequency of any word is inversely proportional to its rank in the frequency table.

As with cities and social networks, so with words. Let’s first download the Brown Corpus from NLTK:

In [33]:
nltk.download('brown')
from nltk.corpus import brown
brown.words()[:10]

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Thang\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.


['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of']

In [34]:
len(brown.words())

1161192

chúng ta có hơn 1 triệu tokens

In [38]:
from collections import Counter
puncs = set((',', '.', '--', '-', '!', '?',\
             ':', ';', '``', "''", '(', ')', '[', ']'))
word_list = (x.lower() for x in brown.words() if x not in puncs)
token_counts = Counter(word_list)
token_counts.most_common(20)

[('the', 69971),
 ('of', 36412),
 ('and', 28853),
 ('to', 26158),
 ('a', 23195),
 ('in', 21337),
 ('that', 10594),
 ('is', 10109),
 ('was', 9815),
 ('he', 9548),
 ('for', 9489),
 ('it', 8760),
 ('with', 7289),
 ('as', 7253),
 ('his', 6996),
 ('on', 6741),
 ('be', 6377),
 ('at', 5372),
 ('by', 5306),
 ('i', 5164)]

A quick glance shows that the word frequencies in the Brown corpus follow the logarithmic relationship Zipf predicted. “The” (rank 1 in term frequency) occurs roughly
twice as often as “of” (rank 2 in term frequency), and roughly three times as often as
“and” (rank 3 in term frequency). If you don’t believe us, use the example code (https://github.com/totalgood/nlpia/blob/master/src/nlpia/book/examples/ch03
_zipf.py) in the nlpia package to see this yourself.
 In short, if you rank the words of a corpus by the number of occurrences and list
them in descending order, you’ll find that, for a sufficiently large sample, the first
word in that ranked list is twice as likely to occur in the corpus as the second word in
the list. And it is four times as likely to appear as the fourth word in the list. So given a
large corpus, you can use this breakdown to say statistically how likely a given word is
to appear in any given document of that corpus. 

# Topic modeling

In [39]:
from nlpia.data.loaders import kite_text, kite_history

ModuleNotFoundError: No module named 'nlpia'

## Return of Zipf

## Relavance ranking

## Tools

Bây giờ có rất nhiều packet hỗ trợ chúng ta tự động hoá. Như là scikit-learn. 

pip install scipy <br/>
pip intall sklearn

Tại đây, bạn có thể sử dụng sklearn để xây dựng một ma trận TF-IDF. Lớp TF-IDF của sklearn là một <i>model</i> và phương thức <i>.transform()</i> cái mà tuân thủ tất cả các nguyên tắc API cho mô hình học máy.

In [41]:
docs

['The faster Harry got to the store, the faster and faster Harry would get home.',
 'Harry is hairy and faster than Jill.',
 'Jill is not as hairy as Harry.']

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = docs
vectorizer = TfidfVectorizer(min_df=1)
model = vectorizer.fit_transform(corpus)
print(model.todense().round(2))

[[0.16 0.   0.48 0.21 0.21 0.   0.25 0.21 0.   0.   0.   0.21 0.   0.64
  0.21 0.21]
 [0.37 0.   0.37 0.   0.   0.37 0.29 0.   0.37 0.37 0.   0.   0.49 0.
  0.   0.  ]
 [0.   0.75 0.   0.   0.   0.29 0.22 0.   0.29 0.29 0.38 0.   0.   0.
  0.   0.  ]]


model TFIDFVectorizer là một ma trận thưa, bởi vì ma trận TF-IDF hầu hết là số 0, hầu hết số documents là nhhor trong số tổng số từ của vocabulary.<br/>
.todense() chuyển đổi một ma trận thưa thành ma trận chính quy.

## Alternatives

Vẽ bảng

## Okapi BM25

Những người thông minh tại Đại học thành phố London đã đưa ra một cách tốt hơn để rank
kết quả tìm kiếm. Thay vì chỉ đơn thuần là tính toán TF-IDF cosin tương đồng, họ bình thường hóa và mịn tương đồng. Họ cũng bỏ qua thuật ngữ trùng lặp trong tài liệu truy vấn, cắt hiệu quả tần số hạn cho các vector truy vấn ở 1. Và dấu chấm
sản phẩm cho tương cosin không bình thường bởi các chỉ tiêu vector TF-IDF (số từ ngữ trong tài liệu và truy vấn), nhưng thay bằng một hàm phi tuyến của
độ dài tài liệu riêng của mình:

Vì vậy, bạn chỉ cần các vectơ cơ bản nhất TF-IDF để thức ăn vào đường ống của bạn để có được
nhà nước-of-the-nghệ thuật trình diễn cho tìm kiếm ngữ nghĩa, phân loại tài liệu, hệ thống thoại, và hầu hết các ứng dụng khác mà chúng tôi đề cập trong chương 1. TF-IDFs là
Giai đoạn đầu tiên trong đường ống của bạn, thiết lập cơ bản nhất của đặc biệt bạn sẽ trích từ văn bản. Trong
chương tiếp theo, chúng tôi tính toán vector chủ đề từ vectơ TF-IDF của bạn. vectơ Topic
là một đại diện tốt hơn về ý nghĩa của nội dung của một túi lời
hơn bất kỳ các vectơ TF-IDF bình thường và làm nhẵn cẩn thận. Và điều duy nhất
trở nên tốt hơn từ đó khi chúng ta chuyển sang vector từ Word2vec trong chương 6 và embeddings ròng thần kinh về ý nghĩa của từ và tài liệu trong chương sau.